## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
import numpy as np
import pandas as pd 
from keras.callbacks import EarlyStopping
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


### Load Dataset Files

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/CNN Assignment 2 /DogBreed_Classification/train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/CNN Assignment 2 /DogBreed_Classification/test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/CNN Assignment 2 /DogBreed_Classification/sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/CNN Assignment 2 /DogBreed_Classification/labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
label = pd.read_csv("/content/labels.csv")

In [0]:
label.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [0]:
label.shape

(10222, 2)

### Print the count of each category of Dogs given in the dataset



In [0]:
label['breed'].value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
brabancon_griffon        67
golden_retriever         67
komondor                 67
briard                   66
eskimo_dog               66
Name: breed, Length: 120, dtype: int64

### Get one-hot encodings of labels

In [0]:
label1  = pd.get_dummies( data = label.breed,drop_first=True)

In [0]:
label1.head()

,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,english_foxhound,english_setter,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
label1.shape

(10222, 119)

In [0]:
# inserting ID as the first columns
label1.insert(0, 'id', label.id)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
from tqdm import tqdm 
import cv2 
x_train = []
y_train = []
for f,i in tqdm(label.values):
    train = cv2.imread('./train/{}.jpg'.format(f),1)
    train_resize = cv2.resize(train, (128,128))
    x_train.append(train_resize)
    y_train.append(i)

100%|██████████| 10222/10222 [00:23<00:00, 440.33it/s]


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
# data is already in 4 dimensions and hence not doing any dimension changes
x_train = np.array(x_train)
y_train = np.array(y_train)

In [0]:
x_train=x_train.reshape(x_train.shape[0],128,128,3).astype('float32')
x_train/=255 

In [0]:
y_train  = pd.get_dummies( y_train,drop_first=False)

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, random_state=242)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
sample = pd.read_csv('/content/sample_submission.csv')

In [0]:
sample2=sample['id']

Run the below code to load the test image files in x_test_feature

In [0]:
x_test = []
i = 0 # initialisation
for f in tqdm(sample2.values): # f for format ,jpg
    #print(f)
    img = cv2.imread('./test/{}.jpg'.format(f),1)
    img_resize = cv2.resize(img, (128, 128)) 
    x_test.append(img_resize)

100%|██████████| 10357/10357 [00:22<00:00, 452.59it/s]


Normalize the test data and convert it into 4 dimensions

In [0]:
x_test = np.array(x_test)
x_test=x_test.reshape(x_test.shape[0],128,128,3).astype('float32')
x_test/=255 

In [0]:
y_test = sample.drop(['id'], axis = 1)

In [0]:
print('Shape of training features : ' , x_train.shape)
print('Shape of training target : ' , y_train.shape)
print('Shape of test features : ' , x_test.shape)
print('Shape of test targets : ' , y_test.shape)
print('Shape of Validation features : ' , x_val.shape)
print('Shape of Validation targets : ' , y_val.shape)

Shape of training features :  (8177, 128, 128, 3)
Shape of training target :  (8177, 120)
Shape of test features :  (10357, 128, 128, 3)
Shape of test targets :  (10357, 120)
Shape of Validation features :  (2045, 128, 128, 3)
Shape of Validation targets :  (2045, 120)


### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=5, padding='same', activation='relu', input_shape=(128,128,3))) 
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(120, activation='softmax'))
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      4864      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      18464     
_________________________________________________________________
flatten (Flatten)            (None, 524288)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               134217984 
_________________________________________________________________
dense_1 (Dense)              (None, 120)               30840     
Total params: 134,272,152
Trainable params: 134,272,152
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
             optimizer='sgd',
             metrics=['accuracy'])

### Use batch_size = 128 and epochs = 10 and execute the model

In [0]:
  model.fit(x_train, y_train ,batch_size=128,nb_epoch=10,verbose=1,validation_data=(x_val, y_val))

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 11s 1ms/sample - loss: 4.7823 - acc: 0.0130 - val_loss: 4.7708 - val_acc: 0.0112
Epoch 2/10
8177/8177 [==============================] - 7s 892us/sample - loss: 4.7527 - acc: 0.0165 - val_loss: 4.7444 - val_acc: 0.0147
Epoch 3/10
8177/8177 [==============================] - 7s 894us/sample - loss: 4.7060 - acc: 0.0210 - val_loss: 4.7064 - val_acc: 0.0200
Epoch 4/10
8177/8177 [==============================] - 7s 895us/sample - loss: 4.6412 - acc: 0.0335 - val_loss: 4.6623 - val_acc: 0.0264
Epoch 5/10
8177/8177 [==============================] - 7s 895us/sample - loss: 4.5524 - acc: 0.0428 - val_loss: 4.6369 - val_acc: 0.0323
Epoch 6/10
8177/8177 [==============================] - 7s 893us/sample - loss: 4.4524 - acc: 0.0514 - val_loss: 4.6175 - val_acc: 0.0298
Epoch 7/10
8177/8177 [==============================] - 7s 894us/sample - loss: 4.3376 - acc: 0.0624 - val_loss: 4.5351 - val

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator


In [0]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    horizontal_flip=True)

In [0]:
x_train1 = x_train

In [0]:
datagen.fit(x_train1)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [0]:
model.fit_generator(datagen.flow(x_train1, y_train, batch_size=32),
                     epochs=10) #steps_per_epoch=len(x_train) / 32

Epoch 1/10
256/256 [==============================] - 27s 107ms/step - loss: 4.7775 - acc: 0.0186
Epoch 2/10
256/256 [==============================] - 28s 108ms/step - loss: 4.5490 - acc: 0.0363
Epoch 3/10
256/256 [==============================] - 27s 107ms/step - loss: 4.3794 - acc: 0.0517
Epoch 4/10
256/256 [==============================] - 28s 108ms/step - loss: 4.2632 - acc: 0.0671
Epoch 5/10
256/256 [==============================] - 28s 110ms/step - loss: 4.1646 - acc: 0.0832
Epoch 6/10
256/256 [==============================] - 29s 112ms/step - loss: 4.0707 - acc: 0.0999
Epoch 7/10
256/256 [==============================] - 28s 109ms/step - loss: 3.9687 - acc: 0.1102
Epoch 8/10
256/256 [==============================] - 28s 108ms/step - loss: 3.8753 - acc: 0.1251
Epoch 9/10
256/256 [==============================] - 28s 108ms/step - loss: 3.7740 - acc: 0.1437
Epoch 10/10
256/256 [==============================] - 28s 109ms/step - loss: 3.6758 - acc: 0.1598


In [0]:
x_train.shape

(8177, 128, 128, 3)

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
base_model= VGG16(weights=('/content/drive/My Drive/CNN Assignment 2 /vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg', input_shape = (128,128,3))

Print the summary of the base_model

In [0]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     


### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
base_model.layers[-2].output

<tf.Tensor 'block5_pool/MaxPool:0' shape=(?, 4, 4, 512) dtype=float32>

In [0]:
x = base_model.layers[-2].output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(120, activation='softmax')(x)

# this is the model we will train
model1 = Model(input=base_model.input, output=predictions)

model1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False


In [0]:
from termcolor import colored    #  python module for ANSII Color formatting for output in terminal
#Check which layers have been frozen 
for layer in model1.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

input_1
False
block1_conv1
False
block1_conv2
False
block1_pool
False
block2_conv1
False
block2_conv2
False
block2_pool
False
block3_conv1
False
block3_conv2
False
block3_conv3
False
block3_pool
False
block4_conv1
False
block4_conv2
False
block4_conv3
False
block4_pool
False
block5_conv1
False
block5_conv2
False
block5_conv3
False
block5_pool
False
flatten_1
True
dense_1
True
dense_2
True
dense_3
True


### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
model1.compile(loss='categorical_crossentropy',
             optimizer='sgd',
             metrics=['accuracy'])

In [0]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_acc', patience=6, mode='auto')
callback_list = [early_stopping]

In [0]:
model1.fit(x_train, y_train ,batch_size=128,validation_data=(x_val, y_val), epochs = 30, callbacks=callback_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 8177 samples, validate on 2045 samples
Epoch 1/30
8177/8177 [==============================] - 11s 1ms/step - loss: 4.7968 - acc: 0.0104 - val_loss: 4.7583 - val_acc: 0.0152
Epoch 2/30
8177/8177 [==============================] - 8s 999us/step - loss: 4.7088 - acc: 0.0303 - val_loss: 4.7069 - val_acc: 0.0298
Epoch 3/30
8177/8177 [==============================] - 8s 999us/step - loss: 4.6285 - acc: 0.0434 - val_loss: 4.6454 - val_acc: 0.0401
Epoch 4/30
8177/8177 [==============================] - 8s 996us/step - loss: 4.5364 - acc: 0.0633 - val_loss: 4.5687 - val_acc: 0.0504
Epoch 5/30
8177/8177 [==============================] - 8s 1ms/step - loss: 4.4291 - acc: 0.0791 - val_loss: 4.4796 - val_acc: 0.0689
Epoch 6/30
8177/8177 [==============================] - 8s 1ms/step - loss: 4.3011 - acc: 0.0972 - val_loss: 4.3834 - val_acc: 0.0753
Epoch 7/30
8177/8177 [=======================

## **Even with VGG16 as feature extractor and dense layers the validation accuracy is very poor and the model is overfitting. **

In [0]:
####  attempt to increase accuracy by adding more trainable dense layers with conv layers as non trainable

In [0]:
x1 = base_model.layers[-3].output
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.25)(x1)
#x1 = Dense(512, activation='relu')(x1)
#x1 = Dropout(0.5)(x1)
x1 = Dense(512, activation='relu')(x1)
x1 = Dropout(0.25)(x1)
#x1 = Dense(256, activation='relu')(x1)
x1 = Dense(256, activation='relu')(x1)
#x1 = Dropout(0.25)(x1)
x1 = Dense(120, activation='relu')(x1)
predictions1 = Dense(120, activation='softmax')(x1)

# this is the model we will train
model2 = Model(input=base_model.input, output=predictions1)

model2.summary()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________

In [0]:
from termcolor import colored    #  python module for ANSII Color formatting for output in terminal
#Check which layers have been frozen 
for layer in model2.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

input_1
False
block1_conv1
False
block1_conv2
False
block1_pool
False
block2_conv1
False
block2_conv2
False
block2_pool
False
block3_conv1
False
block3_conv2
False
block3_conv3
False
block3_pool
False
block4_conv1
False
block4_conv2
False
block4_conv3
False
block4_pool
False
block5_conv1
False
block5_conv2
False
block5_conv3
False
flatten_2
True
dense_4
True
dropout_1
True
dense_5
True
dropout_2
True
dense_6
True
dense_7
True
dense_8
True


In [0]:
#sgd = SGD(lr=0.050, momentum=0.6, nesterov=False)

In [0]:
model2.compile(loss='categorical_crossentropy',
             optimizer='sgd',
             metrics=['accuracy'])

In [0]:
model2.fit(x_train, y_train ,batch_size=128,validation_data=(x_val, y_val), epochs = 20, callbacks=callback_list)

Train on 8177 samples, validate on 2045 samples
Epoch 1/20
8177/8177 [==============================] - 9s 1ms/step - loss: 4.8035 - acc: 0.0081 - val_loss: 4.7865 - val_acc: 0.0142
Epoch 2/20
8177/8177 [==============================] - 9s 1ms/step - loss: 4.7899 - acc: 0.0108 - val_loss: 4.7850 - val_acc: 0.0147
Epoch 3/20
8177/8177 [==============================] - 9s 1ms/step - loss: 4.7858 - acc: 0.0126 - val_loss: 4.7789 - val_acc: 0.0098
Epoch 4/20
8177/8177 [==============================] - 9s 1ms/step - loss: 4.7790 - acc: 0.0139 - val_loss: 4.7735 - val_acc: 0.0132
Epoch 5/20
8177/8177 [==============================] - 9s 1ms/step - loss: 4.7705 - acc: 0.0114 - val_loss: 4.7724 - val_acc: 0.0147
Epoch 6/20
8177/8177 [==============================] - 9s 1ms/step - loss: 4.7666 - acc: 0.0170 - val_loss: 4.7677 - val_acc: 0.0117
Epoch 7/20
8177/8177 [==============================] - 9s 1ms/step - loss: 4.7619 - acc: 0.0149 - val_loss: 4.7611 - val_acc: 0.0171
Epoch 8/20
817

Accuracy is still Poor. 

Will attempt to open up the Conv layers also for training to check if accuracy is improving. 

In [0]:
model3 = Model(input=base_model.input, output=predictions1)
model3.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

In [0]:
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = True

In [0]:
from termcolor import colored    #  python module for ANSII Color formatting for output in terminal
#Check which layers have been frozen 
for layer in model3.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

input_1
True
block1_conv1
True
block1_conv2
True
block1_pool
True
block2_conv1
True
block2_conv2
True
block2_pool
True
block3_conv1
True
block3_conv2
True
block3_conv3
True
block3_pool
True
block4_conv1
True
block4_conv2
True
block4_conv3
True
block4_pool
True
block5_conv1
True
block5_conv2
True
block5_conv3
True
flatten_2
True
dense_4
True
dropout_1
True
dense_5
True
dropout_2
True
dense_6
True
dense_7
True
dense_8
True


In [0]:
model3.compile(loss='categorical_crossentropy',
            optimizer='sgd',
           metrics=['accuracy'])

In [0]:
model3.fit(x_train, y_train ,batch_size=128,validation_data=(x_val, y_val), epochs = 30, callbacks=callback_list)

Train on 8177 samples, validate on 2045 samples
Epoch 1/30
8177/8177 [==============================] - 24s 3ms/step - loss: 4.3929 - acc: 0.0566 - val_loss: 3.8742 - val_acc: 0.1178
Epoch 2/30
8177/8177 [==============================] - 20s 2ms/step - loss: 3.9272 - acc: 0.1090 - val_loss: 3.8145 - val_acc: 0.1218
Epoch 3/30
8177/8177 [==============================] - 20s 2ms/step - loss: 3.5683 - acc: 0.1554 - val_loss: 3.4951 - val_acc: 0.1790
Epoch 4/30
8177/8177 [==============================] - 20s 2ms/step - loss: 3.3009 - acc: 0.1981 - val_loss: 3.1057 - val_acc: 0.2416
Epoch 5/30
8177/8177 [==============================] - 20s 2ms/step - loss: 3.0593 - acc: 0.2442 - val_loss: 2.8964 - val_acc: 0.2743
Epoch 6/30
8177/8177 [==============================] - 20s 2ms/step - loss: 2.8311 - acc: 0.2813 - val_loss: 2.8765 - val_acc: 0.2797
Epoch 7/30
8177/8177 [==============================] - 20s 2ms/step - loss: 2.6105 - acc: 0.3331 - val_loss: 2.6097 - val_acc: 0.3369
Epoch 8

Validation accuracy has slightly improved. But it is still lower. We will increase the training data quantity to see if the model stops overfitting. 

In [0]:
print(x_train.shape)
print(x_test.shape)
Xtrain_new = np.vstack((x_train,x_test))
print(Xtrain_new.shape)

(8177, 128, 128, 3)
(10357, 128, 128, 3)
(18534, 128, 128, 3)


In [0]:
print(y_train.shape)
print(y_test.shape)
ytrain_new = np.vstack((y_train,y_test))
print(ytrain_new.shape)

(8177, 120)
(10357, 120)
(18534, 120)


In [0]:
#model5 = Model(input=base_model.input, output=predictions1)
#model5.summary()

In [0]:
#from termcolor import colored    #  python module for ANSII Color formatting for output in terminal
#Check which layers have been frozen 
#for layer in model4.layers:
 # print (colored(layer.name, 'blue'))
  #print (colored(layer.trainable, 'red'))

In [0]:
model3.fit(Xtrain_new, ytrain_new ,batch_size=128,validation_data=(x_val, y_val), epochs = 30, callbacks=callback_list)

Train on 18534 samples, validate on 2045 samples
Epoch 1/30
18534/18534 [==============================] - 46s 2ms/step - loss: 3.7747 - acc: 0.4285 - val_loss: 2.9753 - val_acc: 0.3971
Epoch 2/30
18534/18534 [==============================] - 43s 2ms/step - loss: 3.5150 - acc: 0.4402 - val_loss: 3.0829 - val_acc: 0.3765
Epoch 3/30
18534/18534 [==============================] - 44s 2ms/step - loss: 3.3914 - acc: 0.4418 - val_loss: 2.9639 - val_acc: 0.3770
Epoch 4/30
18534/18534 [==============================] - 44s 2ms/step - loss: 3.2915 - acc: 0.4429 - val_loss: 2.9633 - val_acc: 0.3756
Epoch 5/30
18534/18534 [==============================] - 43s 2ms/step - loss: 3.2000 - acc: 0.4441 - val_loss: 3.1274 - val_acc: 0.3579
Epoch 6/30
18534/18534 [==============================] - 43s 2ms/step - loss: 3.1270 - acc: 0.4443 - val_loss: 3.0452 - val_acc: 0.3623
Epoch 7/30
18534/18534 [==============================] - 43s 2ms/step - loss: 3.0675 - acc: 0.4445 - val_loss: 3.0618 - val_acc: